In [24]:
import keras
from keras.models import load_model
import random
random.seed(10)
import numpy as np
import math
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam_v2
from IPython.display import clear_output
import time
from pandas_datareader import data
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend as K
import tensorflow as tf
import ta
import random

import statistics

In [25]:
# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [26]:
data = data.DataReader("^DJI",start='2000-10-20',end='2018-12-31',data_source='yahoo')

In [27]:
data.to_csv("csv/DJI.csv")

In [28]:
X1=pd.read_csv('csv/DJI.csv')[0:3600]

In [29]:
X1

Date          High           Low          Open         Close  \
0     2000-10-20  10229.549805  10067.509766  10141.129883  10226.589844   
1     2000-10-23  10361.250000  10216.230469  10230.290039  10271.719727   
2     2000-10-24  10439.309570  10273.570312  10273.570312  10393.070312   
3     2000-10-25  10461.969727  10306.589844  10395.660156  10326.480469   
4     2000-10-26  10454.849609  10265.059570  10326.480469  10380.120117   
...          ...           ...           ...           ...           ...   
3595  2015-02-09  17821.490234  17685.320312  17821.490234  17729.210938   
3596  2015-02-10  17890.339844  17729.240234  17736.150391  17868.759766   
3597  2015-02-11  17897.210938  17759.650391  17867.859375  17862.140625   
3598  2015-02-12  17975.650391  17862.140625  17862.140625  17972.380859   
3599  2015-02-13  18037.410156  17961.759766  17968.650391  18019.349609   

         Volume     Adj Close  
0     319750000  10226.589844  
1     339180000  10271.719727  
2     274990000  10393.070312  
3     312490000  10326.480469  
4     277120000  10380.120117  
...         ...           ...  
3595   81590000  17729.210938  
3596   89930000  17868.759766  
3597   89890000  17862.140625  
3598  117160000  17972.380859  
3599   85230000  18019.349609  

[3600 rows x 7 columns]

In [30]:
Y1=pd.read_csv('csv/DJI.csv')

In [31]:
len(Y1)-50

4526

In [32]:
len(X1)-50

3550

In [33]:
3600/4527

0.7952286282306164

In [34]:
X1.tail()

Date          High           Low          Open         Close  \
3595  2015-02-09  17821.490234  17685.320312  17821.490234  17729.210938   
3596  2015-02-10  17890.339844  17729.240234  17736.150391  17868.759766   
3597  2015-02-11  17897.210938  17759.650391  17867.859375  17862.140625   
3598  2015-02-12  17975.650391  17862.140625  17862.140625  17972.380859   
3599  2015-02-13  18037.410156  17961.759766  17968.650391  18019.349609   

         Volume     Adj Close  
3595   81590000  17729.210938  
3596   89930000  17868.759766  
3597   89890000  17862.140625  
3598  117160000  17972.380859  
3599   85230000  18019.349609

In [35]:
x=X1["Close"][50]

In [36]:
X1['rsi']=ta.momentum.RSIIndicator(X1["Close"], window=14, fillna=True).rsi()
X1['EMA']=ta.trend.EMAIndicator(X1["Close"], window=10, fillna=True).ema_indicator()
X1['ATR']=ta.volatility.average_true_range(high=X1["High"], low=X1["Low"], close=X1["Close"], window=14, fillna=True)

In [37]:
def tech(t):
    ls=[0,0,0]
    #0-hold 1-buy 2-sell
    rsi=X1['rsi'][:t+1] #trading-decision based on RSI
    if(rsi[t]>=70):
        ls[0]=2
    elif (rsi[t]<=30):
         ls[0]=1
    else:
         ls[0]=0
    
    ema=X1['EMA'][:t+1] #trading-decision based on EMA
    if(X1['Low'][t] > ema[t]): #i.e. the ema is lower than the low of the current candle
        ls[1]=2
    elif (X1['High'][t] < ema[t]): #i.e. the ema is higher than the high of the current candle
         ls[1]=1
    else:
         ls[1]=0
  
    atr=X1['ATR'][:t+1] #trading-decision based on ATR
    if(atr[t] > statistics.mean(atr[t-20:t])*1.3): #i.e. the current value of the ATR is at least 30% higher than the average of the last 20 ATR-Values
        ls[2]=2
    elif (atr[t] > statistics.mean(atr[t-20:t])*1.3): #i.e. the current value of the ATR is at least 30% higher than the average of the last 20 ATR-Values
         ls[2]=1
    else:
         ls[2]=0
    return ls

In [38]:
class Portfolio:
    def __init__(self):
        self.cash=1000000
        self.profit=0
        self.inventory = []
        self.nshare=0
        self.last_price=0
        self.sold_share=0
        
    def getData(self):        
        self.vec = []
        self.df=pd.read_csv('csv/DJI.csv')[0:3600]
        return self.df 
    
    def getState(self,data,t):
        self.n=50
        self.res=[]
        for i in range(self.n):                 
            self.res.append(data['Close'][t-i])          
        return np.array([self.res])

In [39]:
class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._model()
        self.target_model = self._model()
        self.update_target_model()
      
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse",optimizer=adam_v2.Adam(learning_rate=0.001))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target,epochs=1, verbose=1)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [40]:
#************************-Training-*********************

rl = RLAgent(50,3)
episode_count=1
for e in range(episode_count):
    agent=Portfolio()
    data=agent.getData()
    data['Action']=""
    data['actual_action']=""
    data['share']=""
    data['Cash']=""
    data['trade_profit']=""
    data['Adj_cash']=""
    l = len(data) - 1
    batch_size = 32
    print( "Episode " + str(e+1) + "/" + str(episode_count))
    state = agent.getState(data, 50)
    initial_cash=agent.cash
    total_profit = 0
    total_brokerage=0
    agent.nshare=0
    agent.last_price=0
    data['Action'][50]=0
    data['share'][50]=agent.nshare
    data['Cash'][50]= agent.cash
    profit=0
    pos=""
   
    for t in range(50,l):
        z=tech(t)
        z0=z.count(0)
        z1=z.count(1)
        z2=z.count(2)
        y=[z0,z1,z2]   #takes all inputs from technical indicators
        z3=y.index(max(y)) # maximum y-value 
        action = rl.act(state)
        data['Action'][t]=0
        data['share'][t]=agent.nshare
        data['Cash'][t]= agent.cash
                
            
        # sit
        next_state = agent.getState(data, t + 1)
        reward = 0
        if action == 0:
            if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
            elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
            elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                    
        elif action == 1:# buy action from deep-q-learning algo
                data['Action'][t]=1
                
                if z3==1:  # loads max y values                  
                    if  agent.nshare == 0: #long buy
                        agent.nshare=int(agent.cash/data['Close'][t])
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Bought-open-long"
                        data['share'][t]=agent.nshare
                        data['Cash'][t]= agent.cash
                    elif agent.nshare < 0: #short buy
                        
                        sell_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((sell_price + data['Close'][t])*(-ns))*0.0010
                        total_brokerage=total_brokerage+brokerage
                        data['trade_profit'][t]=((sell_price - data['Close'][t])*(-ns))-brokerage
                        total_profit = total_profit + ((sell_price - data['Close'][t])*(-ns))-brokerage
                        agent.cash=agent.cash+(-ns*sell_price)+data['trade_profit'][t]
                        data['actual_action'][t]="Bought-closed-short"
                        agent.sold_share=0
                
                    
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    reward = (1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
                else:
                    if agent.nshare == 0:
                        data['Adj_cash'][t]=agent.cash
                    elif agent.nshare >0:
                        data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    elif agent.nshare <0:
                        data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                                                      
        elif action == 2:# sell
            data['Action'][t]=2
            
            if z3==2:  
                    if  agent.nshare == 0: #short sell
                        agent.nshare=int(agent.cash/data['Close'][t])
                             
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.nshare= -agent.nshare
                        agent.sold_share=agent.nshare
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Sold-open-short"
                                
                    elif agent.nshare > 0: 
                        
                        buy_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((buy_price + data['Close'][t])*(ns))*0.0010
                        data['trade_profit'][t]=(data['Close'][t]-buy_price)*(ns)-brokerage
                        total_profit = total_profit + (data['Close'][t]-buy_price)*(ns)-brokerage
                        agent.cash=agent.cash+(ns)*data['Close'][t]-brokerage
                        data['actual_action'][t]="Sold-closed-long"
               
                    
                    data['Adj_cash'][t]=agent.cash+(-agent.nshare)*agent.last_price+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    reward = -1*(1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
            else:
                if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
                elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.nshare)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    
        done = True if t == l - 1 else False
        rl.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
                rl.update_target_model()
                print( "------------------------------------------")
                print ("Initial Cash: " + formatPrice(initial_cash))
                print ("Final Cash: " + formatPrice(data['Adj_cash'][t]))
                print ("Total  Profit: " + formatPrice(data['Adj_cash'][t]-initial_cash))
                print ("Total % Profit:{} ".format((data['Adj_cash'][t]-initial_cash)/initial_cash*100))
                print("-------------------------------------------")
            

        if len(rl.memory) > batch_size:
            rl.replay(batch_size)
 
    if e % 1 == 0:
        rl.model.save("models/trainedmodel" + str(e))
        data.to_csv('csv/after_training.csv')

Episode 1/1


<ipython-input-40-88a4ba9ed576>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Action'][50]=0
<ipython-input-40-88a4ba9ed576>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['share'][50]=agent.nshare
<ipython-input-40-88a4ba9ed576>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Cash'][50]= agent.cash
<ipython-input-40-88a4ba9ed576>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

1/1 [==============================] - 0s 0s/step - loss: 2001.4559


1/1 [==============================] - 0s 0s/step - loss: 11.1275


1/1 [==============================] - 0s 0s/step - loss: 508902.3438


...

1/1 [==============================] - 0s 0s/step - loss: 6716.0073


1/1 [==============================] - 0s 0s/step - loss: 1421.9713


1/1 [==============================] - 0s 16ms/step - loss: 286.9272


1/1 [==============================] - 0s 0s/step - loss: 1814.8248


1/1 [==============================] - 0s 0s/step - loss: 220.5459
------------------------------------------
Initial Cash: $1000000.00
Final Cash: $408305.53
Total  Profit: -$591694.47
Total % Profit:-59.169446875 
-------------------------------------------
1/1 [==============================] - 0s 0s/step - loss: 6.2032
INFO:tensorflow:Assets written to: models/trainedmodel0\assets
